## Проверка работы модели на сторонних данных

Проверка работы модели на данных

1. Диктофон (цифры на англ)

Модель классифицировала такие аудиозаписи как лейбл -1, то есть не смогла распознать цифры на аудиофрагментах. Скорее всего это связано с тем, что условия записи данных сильно отличаются от тех, что были в датассете для обучения. Возможно эту проблему можно решить путем аугментации данных.

2. Номер главы из датасета либри спич.

Модель хорошо предсказывает цифры из датасета либри спич, даже если на фрейме встречается другое слово. 

3. Цифра из МНИСТ датасета + слова из либри спич 

Аналогично предыдущему пункту, модель предиктит верно

## Гипотезы по улучшению

1. Добавить аугментацию данных, а именно: 

    * Шум из датасета QUT-NOISE
    * Аугментация pitch 
    


In [1]:
import pandas as pd
import numpy as np

import os
import sys

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
from IPython.display import Audio

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [4]:
def predict(X):   
    with torch.no_grad():             
        model.eval()     

        output_logits, output_softmax = model(X)
        predictions = torch.argmax(output_softmax,dim=1)        
         
    return  predictions


In [5]:
class Conv1dBlock(nn.Module):
    def __init__(self):
        
        super(Conv1dBlock, self).__init__()        
        kernel_size = 5
        padding = 2
        stride = 1
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=256, kernel_size=kernel_size, padding=2)
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=kernel_size, padding=2)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=kernel_size, padding=2)
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=kernel_size, padding=2)
        self.pool = nn.MaxPool1d(5,2,2)
        self.dropout1 = nn.Dropout2d(0.2)
        self.dropout2 = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(704,32)
        self.fc2 = nn.Linear(32,11)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(self.pool(F.relu(self.conv3(x))))
        x = self.dropout2(self.pool(F.relu(self.conv4(x))))
              
        x = x.view(-1, 64*11)
        x = F.relu(self.fc1(x))
        #x = F.log_softmax(self.fc2(x), dim=1)
        
        
        output_logits = self.fc2(x) 
        
       
        output_softmax = F.log_softmax(output_logits, dim=1)
        
       
        return output_logits, output_softmax  
       

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'{device} selected')

cuda selected


In [7]:
scaler = pickle.load(open('scaler.pkl','rb'))
encoder = LabelEncoder()
encoder.classes_ = np.load('classes.npy')

In [ ]:
model = load_checkpoint('checkpoint.pth')
model.eval() 
model.to(device)

## Wild Audio 

Тестирование модели на голосе записанного с диктофона на телефоне

In [9]:
#10 цифр из примера
Audio('wild_data/wild_audio_1.m4a')

In [10]:
#пример аудиозаписи с одной цифрой из которой получались фичи
Audio('wild_data/wild_audio_1_frame_0.wav')

In [15]:
Features = pd.read_csv('features/wild_features.csv')

X = Features.iloc[: ,:-1].values
PATH = Features['labels'].values

x_test = scaler.transform(X)

In [16]:
x_test = np.expand_dims(x_test, axis=2)
x_test= x_test.reshape(12, 1, 162)
x_test = torch.tensor(x_test, device=device).float() 

pred = predict(x_test)

In [17]:
pred = pred.cpu().numpy()
encoder.inverse_transform(pred)

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

## LibriSpeech

Аудиосэпмл с единицей предскзаан верно

In [23]:
Audio('wild_data/libri_audio_1_frame_0.wav') 

In [24]:
Features = pd.read_csv('features/libri_features.csv')
Features = Features.values.reshape(1,162)
x_test = scaler.transform(Features)

In [25]:
x_test = np.expand_dims(x_test, axis=2)
x_test= x_test.reshape(1, 1, 162)
x_test = torch.tensor(x_test, device=device).float() 

pred = predict(x_test)

In [26]:
pred = predict(x_test)

In [27]:
pred = pred.cpu().numpy()
encoder.inverse_transform(pred)

array([1])

## MNIST Data + Libri SPeech

Аудиосэпмл с единицей предскзаан верно

In [28]:
Audio('wild_data/libri+mnist.wav')

In [29]:
Features = pd.read_csv('features/mnist+libri_features.csv')

In [30]:
Features = Features.values.reshape(1,162)

In [31]:
x_test = scaler.transform(Features)

In [32]:
x_test = np.expand_dims(x_test, axis=2)
x_test= x_test.reshape(1, 1, 162)
x_test = torch.tensor(x_test, device=device).float() 

In [33]:
pred = predict(x_test)

In [34]:
pred = pred.cpu().numpy()
encoder.inverse_transform(pred)

array([1])